# Disaster Image Classification

Made By:
- Austin Kane - 27022229232
- Andreas Immanuel Lukito - 2702211595

In [1]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications import MobileNetV2, EfficientNetB0, EfficientNetB2
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D, Dropout
from tensorflow.keras.models import Model
from tensorflow_addons.metrics import F1Score
from PIL import ImageFile

c:\Users\Austin\Anaconda3\envs\DL\lib\site-packages\tensorflow_addons\utils\tfa_eol_msg.py:23: UserWarning: 

TensorFlow Addons (TFA) has ended development and introduction of new features.
TFA has entered a minimal maintenance and release mode until a planned end of life in May 2024.
Please modify downstream libraries to take dependencies from other repositories in our TensorFlow community (e.g. Keras, Keras-CV, and Keras-NLP). 

For more information see: https://github.com/tensorflow/addons/issues/2807 

  warnings.warn(


## Set the Parameters

In [2]:
IMAGE_SIZE = (224, 224) # MobileNetV2's input size is 224x224 
BATCH_SIZE = 32 
NUM_CLASSES = 11
DATA_DIR = 'Data'

In [3]:
datagen = ImageDataGenerator(
    rescale=1./255, # Normalize the images
    validation_split=0.2, # Train Val Split
    
    # Image Augmentation
    horizontal_flip=True,
    vertical_flip=True,
    rotation_range=20,
    zoom_range=0.2
)

In [4]:
train_generator = datagen.flow_from_directory(
    DATA_DIR,
    target_size=IMAGE_SIZE, # Scale the image to fit model's input
    batch_size=BATCH_SIZE, 
    class_mode='categorical',
    subset='training',
    shuffle=True
)

Found 10943 images belonging to 11 classes.


In [5]:
val_generator = datagen.flow_from_directory(
    DATA_DIR,
    target_size=IMAGE_SIZE, # Scale the image to fit model's input
    batch_size=BATCH_SIZE,
    class_mode='categorical',
    subset='validation',
    shuffle=False
)

Found 2731 images belonging to 11 classes.


## MobileNetV2

In [6]:
base_model = MobileNetV2(
    weights='imagenet',
    include_top=False,
    input_shape=(*IMAGE_SIZE, 3)
)
base_model.trainable = False

In [7]:
x = base_model.output
x = GlobalAveragePooling2D()(x)
x = Dropout(0.3)(x)
x = Dense(128, activation='relu')(x)
x = Dropout(0.3)(x)
predictions = Dense(NUM_CLASSES, activation='softmax')(x)

MobileNet = Model(inputs=base_model.input, outputs=predictions)

In [8]:
f1_metric = F1Score(num_classes=NUM_CLASSES, average='macro')
MobileNet.compile(optimizer='adam',
              loss='categorical_crossentropy',
              metrics=[f1_metric])

In [9]:
ImageFile.LOAD_TRUNCATED_IMAGES = True

In [10]:
history = MobileNet.fit(
    train_generator,
    epochs=10,
    validation_data=val_generator
)

Epoch 1/10
342/342 [==============================] - 138s 398ms/step - loss: 0.7384 - f1_score: 0.5933 - val_loss: 0.4350 - val_f1_score: 0.7470
Epoch 2/10
342/342 [==============================] - 129s 379ms/step - loss: 0.5068 - f1_score: 0.7181 - val_loss: 0.4048 - val_f1_score: 0.7444
Epoch 3/10
342/342 [==============================] - 130s 379ms/step - loss: 0.4497 - f1_score: 0.7517 - val_loss: 0.3695 - val_f1_score: 0.7768
Epoch 4/10
342/342 [==============================] - 129s 377ms/step - loss: 0.4363 - f1_score: 0.7644 - val_loss: 0.3603 - val_f1_score: 0.7664
Epoch 5/10
342/342 [==============================] - 132s 385ms/step - loss: 0.4209 - f1_score: 0.7647 - val_loss: 0.3627 - val_f1_score: 0.7869
Epoch 6/10
342/342 [==============================] - 130s 380ms/step - loss: 0.3929 - f1_score: 0.7864 - val_loss: 0.3409 - val_f1_score: 0.7991
Epoch 7/10
342/342 [==============================] - 129s 378ms/step - loss: 0.3861 - f1_score: 0.7858 - val_loss: 0.3367 -

In [11]:
loss, f1 = MobileNet.evaluate(val_generator)
print(f"Validation F1-Score: {f1:.4f}")

86/86 [==============================] - 26s 300ms/step - loss: 0.3294 - f1_score: 0.7978
Validation F1-Score: 0.7978


In [12]:
from sklearn.metrics import classification_report, accuracy_score, f1_score, precision_score, recall_score
import numpy as np

y_pred_probs = MobileNet.predict(val_generator)
y_pred = np.argmax(y_pred_probs, axis=1)

y_true = val_generator.classes

accuracy = accuracy_score(y_true, y_pred)
precision = precision_score(y_true, y_pred, average='macro')
recall = recall_score(y_true, y_pred, average='macro')
f1 = f1_score(y_true, y_pred, average='macro')

print(f"Accuracy:  {accuracy:.4f}")
print(f"Precision: {precision:.4f}")
print(f"Recall:    {recall:.4f}")
print(f"F1 Score:  {f1:.4f}")

86/86 [==============================] - 28s 312ms/step
Accuracy:  0.8942
Precision: 0.8467
Recall:    0.7966
F1 Score:  0.8163


In [19]:
label_map = val_generator.class_indices

class_names_to_include = ["Damaged_infrastructure", "Damaged_water", "Fire_Urban", "Land_Slide"]
class_indices_to_include = [label_map[name] for name in class_names_to_include]

precision = precision_score(y_true, y_pred, labels=class_indices_to_include, average='macro')
recall = recall_score(y_true, y_pred, labels=class_indices_to_include, average='macro')
f1 = f1_score(y_true, y_pred, labels=class_indices_to_include, average='macro')

print("Evaluation for Damaged_infrastructure, Damaged_water, Fire_Urban, and Land_Slide")
print(f"Precision: {precision:.4f}")
print(f"Recall:    {recall:.4f}")
print(f"F1 Score:  {f1:.4f}")

Evaluation for Damaged_infrastructure, Damaged_water, Fire_Urban, and Land_Slide
Precision: 0.7592
Recall:    0.6751
F1 Score:  0.7011
